In [1]:
import numpy as np
from packages.Utility.CNN.Callbacks import *
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.Gabor import GaborFilterBanks, GaborParams
from packages.Model.CFNet import CFNet3

Using TensorFlow backend.


# Inisialisasi / Persiapan

In [2]:
fold_index = 0
dataset_dir = 'dataset/bin/CF.32.B.30K'
trained_dir = 'trained/CF.32.B.30K CF3.1'

# Memuat Dataset

In [3]:
reader = DatasetReader(dataset_dir)

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold_index)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold_index)

# Inisiasi Callbacks

In [5]:
val_model = ValidateModel(X_train, Y_train_onehot)
save_model = SaveModel(val_model, base_dir = trained_dir, fold = fold_index + 1, min_acc = 0.1)
stop_train = StopTraining(val_model, max_passes = 100)
history_save = HistorySaver(val_model, base_dir = trained_dir, fold = fold_index + 1)

# Parameter Gabor

In [6]:
class ChannelizeGaborParams(GaborParams):
    
    def getParams(self, shape, i, j):
        multiplier = 360 / shape[3]
        degree = multiplier * i
        
        i += 1
        j += 1
        
        lambd = (i * j) / shape[2]
        theta = degree * np.pi / 180
        psi = (i + j) / shape[2]
        sigma = (i + j)
        gamma = (i + j) / shape[2]
        
        return lambd, theta, psi, sigma, gamma

# Mendefinisikan Kernel Gabor

In [7]:
kernel1 = GaborFilterBanks(ChannelizeGaborParams())
kernel2 = GaborFilterBanks(ChannelizeGaborParams())
kernel3 = GaborFilterBanks(ChannelizeGaborParams())
kernel4 = GaborFilterBanks(ChannelizeGaborParams())
kernel5 = GaborFilterBanks(ChannelizeGaborParams())
kernel6 = GaborFilterBanks(ChannelizeGaborParams())

# Paremeter Gabor

In [8]:
class ChannelizeGaborParams(GaborParams):
    
    def getParams(self, shape, i, j):
        multiplier = 360 / shape[3]
        degree = multiplier * i
        
        i += 1
        j += 1
        
        lambd = (i * j) / shape[2]
        theta = degree * np.pi / 180
        psi = (i + j) / shape[2]
        sigma = (i + j)
        gamma = (i + j) / shape[2]
        
        return lambd, theta, psi, sigma, gamma

# Mendefinisikan Kernel

In [6]:
kernel1 = GaborFilterBanks(ChannelizeGaborParams())
kernel2 = GaborFilterBanks(ChannelizeGaborParams())
kernel3 = GaborFilterBanks(ChannelizeGaborParams())
kernel4 = GaborFilterBanks(ChannelizeGaborParams())
kernel5 = GaborFilterBanks(ChannelizeGaborParams())
kernel6 = GaborFilterBanks(ChannelizeGaborParams())

# Mendefinisikan Model

In [9]:
ars = CFNet3(kernels = {
    1: kernel1,
    2: kernel2,
    3: kernel3,
    4: kernel4,
    5: kernel5,
    6: kernel6
}, loss = 'categorical_crossentropy', optimizer = 'adadelta')
ars.save(f'{trained_dir}/Fold {fold_index + 1}/init.h5')

model = ars.model
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c1 (Conv2D)                  (None, 32, 32, 32)        896       
_________________________________________________________________
c2 (Conv2D)                  (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
c3 (Conv2D)                  (None, 16, 16, 64)        18496     
_________________________________________________________________
c4 (Conv2D)                  (None, 16, 16, 64)        36928     
_________________________________________________________________
mp4 (MaxPooling2D)   

# Pelatihan

In [9]:
ars.compile()
history = model.fit(X_train, Y_train_onehot, epochs = 100, verbose = 1, batch_size = 128, callbacks=[val_model, save_model, history_save, stop_train], initial_epoch = 0)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
22500/22500 [==============================] - 125s 6ms/step - loss: 2.2999 - acc: 0.1718
New best accuracy : 0.25897777777777775
Model saved to disk
Passes : 0/100
Epoch 2/100
22500/22500 [==============================] - 122s 5ms/step - loss: 2.1673 - acc: 0.2193
New best accuracy : 0.26222222222222225
Model saved to disk
Passes : 0/100
Epoch 3/100
22500/22500 [==============================] - 125s 6ms/step - loss: 2.0988 - acc: 0.2428
New best accuracy : 0.2638666666666667
Model saved to disk
Passes : 0/100
Epoch 4/100
22500/22500 [==============================] - 119s 5ms/step - loss: 2.0585 - acc: 0.2572
New best accuracy : 0.2936888888888889
Model saved to disk
Passes : 0/100
Epoch 5/100
22500/22500 [==============================] - 119s 5ms/step - loss: 2.0429 - acc: 0.2706
Accuracy : 0.2736
Model saved to disk
Passes : 1/100
Epoch 6/100
22500/22500 [==============================] - 119s 5ms/step - loss: 2.0176 - a